In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.utils import AnalysisException
from spark_utils import iniciar_spark, fechar_spark

import utils
import os

spark = iniciar_spark("ingestaoSilver")

Sessão Spark iniciada com o nome: ingestaoSilver


In [11]:
# Variaveis
layer_bronze = "bronze"
bronze_path = utils.ler_configuracoes(layer_bronze)
print(bronze_path)
layer_silver = "silver"
silver_path = utils.ler_configuracoes(layer_silver)
print(silver_path)

directory = os.path.join(os.path.dirname(os.path.abspath("../../")), bronze_path)
print(directory)

dados_cervejarias\data_lake\bronze
dados_cervejarias\data_lake\silver\brewery
c:\Users\wilso\Downloads\Desafio-BEES---Caso-Du\dados_cervejarias\data_lake\bronze


In [12]:
file = utils.get_latest_folder_with_file(directory)

df_bronze = spark.read.format("json") \
    .option("inferSchema", "true") \
    .option("multiline", "true") \
    .load(file)

if df_bronze is None:
    print("DataFrame bronze é None. Abortando ingestão silver.")

print("Schema do DataFrame bronze:")
df_bronze.printSchema()
print(f"Número de linhas no DataFrame bronze: {df_bronze.count()}")
df_bronze.show(truncate=False)

Schema do DataFrame bronze:
root
 |-- address_1: string (nullable = true)
 |-- address_2: string (nullable = true)
 |-- address_3: string (nullable = true)
 |-- brewery_type: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- id: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- name: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_province: string (nullable = true)
 |-- street: string (nullable = true)
 |-- website_url: string (nullable = true)

Número de linhas no DataFrame bronze: 50
+---------------------------+---------+---------+------------+--------------+-------------+------------------------------------+-----------------+------------------+-------------------------------+------------+-----------+-------------+--------------+---------------------------+---------

In [13]:
df_bronze = df_bronze.select(
    F.col("id").alias("brewery_id"),
    F.col("name").alias("brewery_name"),
    F.col("brewery_type"),
    F.concat(F.col("street"), F.lit(" "), F.col("address_1")).alias("address"),
    F.col("city"),
    F.col("state_province").alias("state"),
    F.col("postal_code"),
    F.col("country"),
    F.col("phone"),
    F.col("website_url"),
    F.col("longitude"),
    F.col("latitude")
).distinct()

In [17]:

if silver_path is None:
    print("Caminho silver não configurado. Abortando ingestão silver")

silver_path = os.path.join(os.path.dirname(os.path.abspath("../../")), silver_path)
print(silver_path)

c:\Users\wilso\Downloads\Desafio-BEES---Caso-Du\dados_cervejarias\data_lake\silver\brewery


In [8]:
# target_path = os.path.join(silver_path, "brewery\\")

In [15]:
file_name = os.path.join(target_path)
try:
    df_bronze.write.mode("overwrite").parquet(file_name)
    print(f"DataFrame successfully saved as Parquet in: {file_name}")
except Exception as e:
    print(f"Error: {e}")

DataFrame successfully saved as Parquet in: c:\Users\wilso\Downloads\Desafio-BEES---Caso-Du\dados_cervejarias\data_lake\silver\brewery\brewery\
